## Configurations

In [ ]:
import sys
sys.path.append('/analytics/shared/packages/')
import os
import csv
import traceback
def prvar(__x):
    print traceback.extract_stack(limit=2)[0][3][6:][:-1],"=",__x

import numpy as np
import pandas as pd
import math
from math import sqrt
import random
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error
from subprocess import check_output
from sklearn.model_selection import train_test_split



# Data Preparation Functions

## Alias names for CSV files

In [ ]:
Input_folder = 'data_files'
DE_ratings_file = 'expert_rating.csv' #CSV file containing domain expert ratings
S_rating_file = 'student_ratings.csv' #CSV file containing student ratings
S_performance_file = 'student_performance.csv' #CSV file containing student performance level obtained from their scores in the course
q_difficulty_file = 'question_difficulty.csv' # CSV file containing question difficulty and question ID

In [ ]:
def load_data(Input_folder, data_file):
    '''
    load data sets into pandas data frames
    '''
    filePath = Input_folder + '/' + data_file
    df = pd.read_csv(filePath, encoding='UTF-8')
    return df


def by_performance_ratings(performane_df, quality_rating_df):
    '''
    merging high_perfroming student_performance and 
    quality ratings dataframes to have ratings based on students performance
    
    returns 3 dataframes containing ratings given by each group of students.
    '''
    by_performane_rating_df = pd.merge(quality_rating_df, performane_df, on=['UserID'])
    ratings_and_high_pf = by_performane_rating_df[by_performane_rating_df.Performance_group == 'High-performing']
    ratings_and_average_pf = by_performane_rating_df[by_performane_rating_df.Performance_group == 'Average-performing']
    ratings_and_low_pf = by_performane_rating_df[by_performane_rating_df.Performance_group == 'Low-performing']
    return ratings_and_high_pf, ratings_and_average_pf, ratings_and_low_pf 


def by_performance_rating_aggregation(by_performane_rating_df, group_mean, group_std):    
    '''
    Applies mean aggregation on the ratings given by each group of students
    '''
    by_performance_effectiveness_stats = by_performane_rating_df.groupby(['QID'])['effectiveness'].agg([np.mean, np.std])
    by_performance_effectiveness_stats = by_performance_effectiveness_stats.reset_index()
    by_performance_effectiveness_stats = by_performance_effectiveness_stats.rename(columns={'mean': group_mean, 'std': group_std})
    return by_performance_effectiveness_stats


def merge_all_student_ratings(effectiveness_rating_and_high_performing, effectiveness_rating_and_average_performing, effectiveness_rating_and_low_performing, quality_ratings_df, Study_Answers_diff):
    '''
    Calls the aggregation function on the ratings given by each group of students
    and then merges the aggregation results to form a uniform data set
    '''
    high_performing_effectiveness_stats = by_performance_rating_aggregation(effectiveness_rating_and_high_performing, 'High-mean', 'High-std')
    average_performing_effectiveness_stats = by_performance_rating_aggregation(effectiveness_rating_and_average_performing, 'Average-mean', 'Average-std')
    low_performing_effectiveness_stats = by_performance_rating_aggregation(effectiveness_rating_and_low_performing, 'Low-mean', 'Low-std')
    ratings_eff_class_df = by_performance_rating_aggregation(quality_ratings_df, 'Class-mean', 'Class-std')
    
    students_effectiveness_ratings = pd.merge(low_performing_effectiveness_stats, high_performing_effectiveness_stats, on=['QID'])
    students_effectiveness_ratings = pd.merge(students_effectiveness_ratings, average_performing_effectiveness_stats, on=['QID'])
    students_effectiveness_ratings = pd.merge(students_effectiveness_ratings, ratings_eff_class_df, on=['QID'])
    students_effectiveness_ratings = pd.merge(students_effectiveness_ratings, Study_Answers_diff, on=['QID'])

    return students_effectiveness_ratings


def create_complete_dataframe(students_effectiveness_ratings, experts_ratings):
    '''
    This function merges the dataframe contaning the given 
    ratings by each group of students to each question, with 
    the the dataframe containing given ratings by domain experts 
    to each question
    '''
    return pd.merge(students_effectiveness_ratings, experts_ratings[['QID', 'Expert Ratings', 'Expert -std']], on=['QID'])


def by_bin_ratings(QIDS_by_bin, ratings):
    '''
    merges ratings given by each group of students to each bin of resources
    '''
    by_bin_ratings = pd.merge(QIDS_by_bin, ratings, on = ['QID'])
    return by_bin_ratings

In [ ]:
# load student ratings, student performnce and expert ratings into 3 data frames
quality_ratings_df = load_data(Input_folder, S_rating_file) 
performance_df = load_data(Input_folder, S_performance_file)
expert_ratings_df = load_data(Input_folder, DE_ratings_file)
question_difficulty_df = load_data(Input_folder, q_difficulty_file)

# loading questions into different dataframes based on their quality
eff_qbin_df = question_difficulty_df[['QID', 'question_quality']][question_difficulty_df['question_quality'] == 'high_quality']
sweff_qbin_df = question_difficulty_df[['QID', 'question_quality']][question_difficulty_df['question_quality'] == 'average_quality']
ineff_qbin_df = question_difficulty_df[['QID', 'question_quality']][question_difficulty_df['question_quality'] == 'low_quality']



# divide ratings by performance
ratings_and_high_pf_df, ratings_and_average_pf_df, ratings_and_low_pf_df = by_performance_ratings(performance_df, quality_ratings_df)

# creating a data frame containing ratings given by each group of students to each question  
students_quality_ratings_df = merge_all_student_ratings(ratings_and_high_pf_df, ratings_and_average_pf_df, ratings_and_low_pf_df, quality_ratings_df, question_difficulty_df)

#merging student rating and domain expert ratings
full_quality_ratings_df = create_complete_dataframe(students_quality_ratings_df, expert_ratings_df)


# ratings given by each group of students to each bin of questions
effective_question_ratings = by_bin_ratings(eff_qbin_df, full_quality_ratings_df)
effective_question_ratings[['Low-mean', 'Low-std', 'High-mean', 'High-std', 'Average-mean', 'Average-std', 'Expert Ratings' , 'Expert -std', 'Difficulty']].agg([np.mean])

somewhat_effective_question_ratings = by_bin_ratings(sweff_qbin_df, full_quality_ratings_df)
somewhat_effective_question_ratings[['Low-mean', 'Low-std', 'High-mean', 'High-std', 'Average-mean', 'Average-std', 'Expert Ratings' , 'Expert -std', 'Difficulty']].agg([np.mean])

ineffective_question_ratings = by_bin_ratings(ineff_qbin_df, full_quality_ratings_df)
ineffective_question_ratings[['Low-mean', 'Low-std', 'High-mean', 'High-std', 'Average-mean', 'Average-std', 'Expert Ratings' , 'Expert -std', 'Difficulty']].agg([np.mean])

# End of Data Peparation Functions

# Evaluations

# Data-Driven Evaluations

## Analysis 1: Rating Aggregations

This analysis is removed for the sake of data privacy

### Student performance (mean and SD in each group:high, average, low)

In [ ]:
# This analysis is commentet for the sake of data privacy

# [high_performing_mark_mean, high_performing_mark_std] = high_performing_df['Mark'].agg([np.mean, np.std])
# [av_performing_mark_mean, av_performing_mark_std] = average_performing_df['Mark'].agg([np.mean, np.std])
# [low_performing_mark_mean, low_performing_mark_std] = low_performing_df['Mark'].agg([np.mean, np.std])

In [ ]:
# prvar(high_performing_mark_mean)
# prvar(high_performing_mark_std)
# prvar(av_performing_mark_mean)
# prvar(av_performing_mark_std)
# prvar(low_performing_mark_mean)
# prvar(low_performing_mark_std)

### Who is more willing to rate?

#### Total and average activity (answering question) by each group of students on all questions on the platform

In [ ]:
# [act_high_tot, act_high_mean, act_high_std] = high_performing_df['ActivityCount'].agg([np.sum, np.mean, np.std])
# [act_av_tot, act_av_mean, act_av_std] = average_performing_df['ActivityCount'].agg([np.sum, np.mean, np.std])
# [act_low_tot, act_low_mean, act_low_std] = low_performing_df['ActivityCount'].agg([np.sum, np.mean, np.std])

In [ ]:
# prvar(act_high_tot)
# prvar(act_high_mean)
# prvar(act_av_tot)
# prvar(act_av_mean)
# prvar(act_low_tot)
# prvar(act_low_mean)

#### Total and average ratings by each group of students

In [ ]:
# h_performing_ratings = pd.merge(high_performing_df, Question_ratings, on = ['UserID'])
# av_performing_ratings = pd.merge(average_performing_df, Question_ratings, on = ['UserID'])
# l_performing_ratings = pd.merge(low_performing_df, Question_ratings, on = ['UserID'])

In [ ]:
# rating_high_tot = len(h_performing_ratings)
# rating_high_mean = len(h_performing_ratings)/high_performing_df['UserID'].nunique()

# rating_av_tot = len(av_performing_ratings)
# rating_av_mean = len(av_performing_ratings)/average_performing_df['UserID'].nunique()

# rating_low_tot = len(l_performing_ratings)
# rating_low_mean = len(l_performing_ratings)/low_performing_df['UserID'].nunique()

In [ ]:
# prvar(rating_high_tot)
# prvar(rating_high_mean)
# prvar(rating_av_tot)
# prvar(rating_av_mean)
# prvar(rating_low_tot)
# prvar(rating_low_mean)

In [ ]:
# a = av_performing_ratings['UserID'].value_counts()
# prvar(np.mean(a.values))
# prvar(np.std(a.values))

### Distribution of ratings by each group

In [ ]:
# boxplot = full_effectiveness_ratings.boxplot(column=['Low-mean', 'High-mean', 'Average-mean', 'Expert Ratings'], grid=False)

## Analysis 2: Error Analysis 
## +
## Analysis 4: Time Analysis

# RMSE

## RMSE (Static)

In [ ]:
rmse_h = sqrt(mean_squared_error(full_quality_ratings_df['High-mean'], full_quality_ratings_df['Expert Ratings']))
rmse_l = sqrt(mean_squared_error(full_quality_ratings_df['Low-mean'], full_quality_ratings_df['Expert Ratings']))
rmse_av = sqrt(mean_squared_error(full_quality_ratings_df['Average-mean'], full_quality_ratings_df['Expert Ratings']))
rmse_class = sqrt(mean_squared_error(full_quality_ratings_df['Class-mean'], full_quality_ratings_df['Expert Ratings'])) 

In [ ]:
prvar(rmse_h)
prvar(rmse_l)
prvar(rmse_av)
prvar(rmse_class)

## Error over time (RMSE over time)

### high_performing students and experts

In [ ]:
def weekly_rmse_func(ratings_ot_df, expert_ratings):
    #converting the create_at column of the dataframe to a meaningful datetime type for python
    ratings_df = ratings_ot_df.copy()
    ratings_df['date'] = ratings_df['created_at']
#     ratings_df['date'] = pd.to_datetime(ratings_df.date, format='%d/%m/%Y') 
    ratings_df['date'] = pd.to_datetime(ratings_df['date']) 
    
    #tag each interaction by the week it was happened
    ratings_df['week_number_of_year'] = ratings_df['date'].dt.week
    
    #sorting values based on the week of the year 
    ratings_df = ratings_df.sort_values(['week_number_of_year'])
    
    # 1- getting the unique number of weeks in the dataset
    weeks = ratings_df['week_number_of_year'].unique()
    weeks.sort()
    
    prvar(weeks)
    
    weekly_RMSE = []
    for week in weeks:
        temp_df = pd.DataFrame(columns = ratings_df.columns)
        temp_df = ratings_df.loc[(ratings_df.week_number_of_year == week)]
    

        # average question efectiveness in each week and convert the findings into a dataframe
        temp_df = pd.merge(temp_df, expert_ratings, on=['QID'])
        rmse = sqrt(mean_squared_error(temp_df['Expert Ratings'], temp_df['effectiveness']))
        weekly_RMSE.append([week-29, rmse])
    
    # removing the last two weeks as they are inclluded in the 13 weeks duration of the semester
    weekly_RMSE.remove(weekly_RMSE[-1])
    weekly_RMSE.remove(weekly_RMSE[-1])
    return weekly_RMSE

In [ ]:
hperforming_weekly_RMSE = weekly_rmse_func(ratings_and_high_pf_df, expert_ratings_df)
avperforming_weekly_RMSE = weekly_rmse_func(ratings_and_average_pf_df, expert_ratings_df)
lperforming_weekly_RMSE = weekly_rmse_func(ratings_and_low_pf_df, expert_ratings_df)


week, h_weekly_rmse = map(list, zip(*hperforming_weekly_RMSE))
week, l_weekly_rmse = map(list, zip(*lperforming_weekly_RMSE))
week, avg_weekly_rmse = map(list, zip(*avperforming_weekly_RMSE))

In [ ]:
# fig = plt.figure(figsize=(18,6))
# # fig.subplots_adjust(hspace=0.4, wspace=0.4)

# plt.subplot(1,3,1)
# x = np.arange(13) + 1
# prvar(x)
# y_h = h_weekly_rmse
# prvar(y_h)
# y_avg = avg_weekly_rmse
# y_l = l_weekly_rmse
# slope, intercept, r_value, p_value, std_err = stats.linregress(x,y_h)
# prvar(p_value)
# prvar(r_value)
# line_h = slope*x+intercept
# f1 = plt.scatter(list(x), list(y_h), c ='black', label='RMSE', s = 65)
# l1 = plt.plot(x, line_h, color = 'black', label='Fitted line')
# plt.xlabel('High performing', fontsize=14)
# plt.ylabel('Weekly RMSE', fontsize=14)
# plt.legend()


# plt.subplot(132)
# slope, intercept, r_value, p_value, std_err = stats.linregress(x,y_avg)
# line_avg = slope*x+intercept

# plt.scatter(list(x), list(y_avg), c ='black', label='RMSE', s = 65)
# plt.plot(x, line_avg, color = 'black', label='Fitted line')
# plt.xlabel('Average performing', fontsize=14)
# plt.legend()
# prvar(p_value)
# prvar(r_value)


# plt.subplot(133)
# slope, intercept, r_value, p_value, std_err = stats.linregress(x,y_l)
# line_l = slope*x+intercept

# plt.scatter(list(x), list(y_l), label='RMSE',  c ='black' , s= 65)
# plt.plot(x, line_l, color = 'black', label='Fitted line')
# plt.xlabel('Low performing', fontsize=14)
# plt.legend()
# plt.show()
# prvar(p_value)
# prvar(r_value)


## Analysis 3: Regression Analysis

In [ ]:
fig = plt.figure(figsize=(18,6))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
cm = plt.cm.get_cmap('RdYlBu')
plt.subplot(1,3,1)
n = full_quality_ratings_df['QID'].values 
x_h = (full_quality_ratings_df['High-mean']).values
y_h = (full_quality_ratings_df['Expert Ratings']).values
z_h = full_quality_ratings_df['Difficulty'].values
z_h = (1 - z_h) 
z_h = list(z_h)
slope, intercept, r_value, p_value, std_err = stats.linregress(x_h,y_h)
line_h = slope*x_h+intercept
prvar(p_value)
prvar(r_value)
t_h = z_h
f1 = plt.scatter(list(x_h), list(y_h), label='Data', c =t_h, cmap = 'coolwarm', s = 65)
plt.xticks(np.arange(2,5, 0.5))
l1 = plt.plot(x_h, line_h, color = 'black', label='Fitted line')
plt.xlabel('High performing', fontsize=14)
plt.ylabel('Domain Experts', fontsize=14)
plt.legend()


plt.subplot(132)

x_avg = (full_quality_ratings_df['Average-mean']).values
y_avg = (full_quality_ratings_df['Expert Ratings']).values
z_avg = full_quality_ratings_df['Difficulty'].values
z_avg = (1 - z_avg) 
z_avg = list(z_avg)
t_avg = z_avg
slope, intercept, r_value, p_value, std_err = stats.linregress(x_avg,y_avg)
prvar(p_value)
prvar(r_value)
line_avg = slope*x_avg+intercept
f1 = plt.scatter(list(x_avg), list(y_avg), label='Data', c =t_avg, cmap = 'coolwarm', s = 65)

plt.xticks(np.arange(2,5, 0.5))
plt.plot(x_avg, line_avg, color = 'black', label='Fitted line')
plt.xlabel('Average performing', fontsize=14)
plt.legend()
# plt.show()



plt.subplot(133)

x_l = (full_quality_ratings_df['Low-mean']).values
y_l = (full_quality_ratings_df['Expert Ratings']).values
z_l = full_quality_ratings_df['Difficulty'].values
z_l = (1 - z_l) 
z_l = list(z_l)
t_l = z_l 
slope, intercept, r_value, p_value, std_err = stats.linregress(x_l,y_l)
line_l = slope*x_l+intercept
prvar(p_value)
prvar(r_value)
f1 = plt.scatter(list(x_l), list(y_l), label='Data', c =t_l, cmap = 'coolwarm', s = 65)

plt.xticks(np.arange(2,5, 0.5))
plt.plot(x_l, line_l, color = 'black', label='Fitted line')
plt.xlabel('Low performing', fontsize=14)
plt.legend()
plt.colorbar(f1)

plt.show()


In [ ]:
full_quality_ratings_df

In [ ]:
fig = plt.figure(figsize=(18,6))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
cm = plt.cm.get_cmap('RdYlBu')
plt.subplot(1,4,1)
x_h = (full_quality_ratings_df['Class-mean']).values
y_h = (full_quality_ratings_df['Expert Ratings']).values
z_h = full_quality_ratings_df['Difficulty'].values
z_h = (1 - z_h) * 150
z_h = list(z_h)
slope, intercept, r_value, p_value, std_err = stats.linregress(x_h,y_h)
line_h = slope*x_h+intercept
prvar(p_value)
prvar(r_value)

# f1 = plt.scatter(list(x_h), list(y_h), s = z_h, label='Data', c ='r')
plt.scatter(list(x_h), list(y_h), label='Ratings',  c ='black')
# plt.colorbar(f1)
plt.xticks(np.arange(2,5, 0.5))
l1 = plt.plot(x_h, line_h, color = 'black', label='Fitted line')
plt.xlabel('Class', fontsize=18)
plt.ylabel('Domain Experts', fontsize=18)
# plt.legend(fontsize=12)
# plt.show()


plt.subplot(142)

x_avg = (full_quality_ratings_df['High-mean']).values
y_avg = (full_quality_ratings_df['Expert Ratings']).values
z_avg = full_quality_ratings_df['Difficulty'].values
z_avg = (1 - z_avg) * 150
z_avg = list(z_avg)

slope, intercept, r_value, p_value, std_err = stats.linregress(x_avg,y_avg)
prvar(p_value)
prvar(r_value)
line_avg = slope*x_avg+intercept

# plt.scatter(list(x_l), list(y_l), s = z_l, label='Data', c ='g')
plt.scatter(list(x_avg), list(y_avg), c ='black', label='Ratings')
plt.xticks(np.arange(2,5, 0.5))
plt.plot(x_avg, line_avg, color = 'black', label='Fitted line')
plt.xlabel('High-performing', fontsize=18)
# plt.legend(fontsize=12)
# plt.show()

plt.subplot(143)

x_avg = (full_quality_ratings_df['Average-mean']).values
y_avg = (full_quality_ratings_df['Expert Ratings']).values
z_avg = full_quality_ratings_df['Difficulty'].values
z_avg = (1 - z_avg) * 150
z_avg = list(z_avg)

slope, intercept, r_value, p_value, std_err = stats.linregress(x_avg,y_avg)
prvar(p_value)
prvar(r_value)
line_avg = slope*x_avg+intercept

# plt.scatter(list(x_l), list(y_l), s = z_l, label='Data', c ='g')
plt.scatter(list(x_avg), list(y_avg), c ='black', label='Ratings')
plt.xticks(np.arange(2,5, 0.5))
plt.plot(x_avg, line_avg, color = 'black', label='Fitted line')
plt.xlabel('Average-performing', fontsize=18)
# plt.legend(fontsize=12)
# plt.show()


plt.subplot(144)

x_l = (full_quality_ratings_df['Low-mean']).values
y_l = (full_quality_ratings_df['Expert Ratings']).values
z_l = full_quality_ratings_df['Difficulty'].values
z_l = (1 - z_l) * 150
z_l = list(z_l)

slope, intercept, r_value, p_value, std_err = stats.linregress(x_l,y_l)
line_l = slope*x_l+intercept
prvar(p_value)
prvar(r_value)
# plt.scatter(list(x_l), list(y_l), s = z_l, label='Data', c ='g')
plt.scatter(list(x_l), list(y_l), label='Ratings',  c ='black' )
plt.xticks(np.arange(2,5, 0.5))
plt.plot(x_l, line_l, color = 'black', label='Fitted line')
plt.xlabel('Low-performing', fontsize=18)
# plt.legend(fontsize=12)
plt.show()
# plt.scatter(list(x_l), list(y_l), label='ratings', c ='b')

# plt.plot(x_l, line_l, label='Fitted line', color = 'black', ls = '--')
# plt.xlabel('Subject Matter Experts', fontsize=14)
# plt.ylabel('Low-performing', fontsize=14)
# plt.legend()
# plt.show()

## End of Data-Driven Evaluations

# Hybrid Approach

### Geneal funbctions for creating a student by question rating matrix

In [ ]:
'''
creates a dictionary in which user ids are map to numbers: uDict
'''
def mapUsersToNumbers(rating_df):
    map = {}
    current =0
    for index, row in rating_df.iterrows():
        user = row['UserID']
        if map.has_key(user)==False:
            map[user]=current
            current = current+1
    return map, current 

'''
creates a dictionary in which question names is mapped as distinct question name to dictionary: qDict
'''
def mapQuestionsToNumbers(rating_df):
    map = {}
    current =0
    for index, row in rating_df.iterrows():
        question = row['QID']
        if map.has_key(question)==False:
            map[question]=current
            current = current+1
    return map, current


'''
creates QT matrice: rows: question id, columns:topic id -> QMAT is oprganized based on qDict and tDict
'''
def createTMatrix(qDict, qSize, uDict, uSize, rating_df):
    QTMat = np.zeros((qSize, uSize))
    for index, row in rating_df.iterrows(): 
        qid = row['QID']
        uid = row['UserID']
        rating = row['effectiveness']
        if qDict.has_key(qid)==True and uDict.has_key(uid)==True:
            QTMat[qDict[qid]][uDict[uid]] =rating
    return QTMat 


def createResponse_matrix(qDict, qSize, ex_rating_df):
    QRMat = np.zeros((qSize, 1))
    for index, row in ex_rating_df.iterrows(): 
        qid = row['QID']
        rating = row['Expert Ratings']
        if qDict.has_key(qid)==True:
            QRMat[qDict[qid]] = rating
        
    return QRMat



# writes matrix R into PathName/FileName
def WriteMatrixToTable(PathName, FileName, R, uDict, qDict, headings):
    '''
    Writes the matrix R into PathName/FileName
    '''
    if not os.path.exists(PathName):   
        os.makedirs(PathName)
        
    FilePath = PathName + "/" + FileName
    fr = open(FilePath, 'wb')
    try:
        writer = csv.writer(fr)
        if headings:
            writer.writerow(headings)
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] <> 0:
                    writer.writerow([i, j, R[i][j]])
    finally:
        fr.close() 


def load(pathName, fileName, uDict, uSize, add_info, qDict, qSize, delimiter ):
    '''
    loads data from CSV file into a matrix
    '''
    train_set = np.zeros((uSize+add_info, qSize)) # add_info is added to handle different scenarios in the experiments
    qfile = pathName + "/" + fileName
    csv_file = csv.reader(open(qfile, "rU"), delimiter=delimiter)
    i =0;
    for row in csv_file:
        id1 = int(row[0])
        id2  = int(row[1])
        rating = row[2]
        train_set[id1][id2] = rating
    return train_set


def createTMatrix_reputation(qDict, qSize, rating_df):
    '''
    create a matrix based on the average of ratings 
    by each group of students and the question dictionary.
    This matrix would be used by MF approaches that are 
    based on performance
    '''
    QTMat = np.zeros((qSize, 1))
    for index, row in rating_df.iterrows(): 
        qid = row['QID']
        h_rating = row['High-mean']
        avg_rating = row['Average-mean']
        l_rating = row['Low-mean']
        if qDict.has_key(qid)==True:
            QTMat[qDict[qid]][0] = h_rating
#             QTMat[qDict[qid]][1] = avg_rating
#             QTMat[qDict[qid]][2] = l_rating
    return QTMat 


In [ ]:
uDict, uSize = mapUsersToNumbers(quality_ratings_df) # student dictionary
qDict, qSize = mapQuestionsToNumbers(quality_ratings_df) # question dictionary
QTMat = createTMatrix(qDict, qSize, uDict, uSize, quality_ratings_df)
QRMat = createResponse_matrix(qDict, qSize, expert_ratings_df) # expert ratings based on the question dictionary

QTMat_mean = np.true_divide(QTMat.sum(1),(QTMat!=0).sum(1)).reshape(-1, 1) # the average of ratings given by all students
QTMat_reputation = createTMatrix_reputation(qDict, qSize, full_quality_ratings_df) # the average ratings given by each group of students

### MF with MyMediaLite

In [ ]:
inputfolder = 'recoms'
recomfolder = 'recoms'
trainFile = "train.csv"
testFile = "test.csv"
recommender='BiasedMatrixFactorization'
predictionFile = 'predictions.csv'

In [ ]:
def  RecSys(inputfolder, recomfolder, recommender, trainset, testset, output):
    '''
    recommender system with MyMediaLite
    '''
    executable = "rating_prediction"

    check_output(
                ['mono', executable,
#                 '--recommender-options', "num_iter=1000",
#                 '--recommender-options', "num_factors=5",
                '--training-file', inputfolder + '/' + trainset,
                '--test-file', inputfolder + '/'+ testset,
                '--recommender', recommender,            
                '--prediction-file' , recomfolder + '/' + output])



## The two unsupervised MF-based approaches

In [ ]:
##########################################################
### Matrix factorization with K-fold Cross validation  ###
##########################################################

def spilt_data_cv(QTMat, QTMat_extra,  QRMat, uDict, qDict, indices, mode):
    '''
    mode: students -> only ratings from students and their mean or only ratings from g=high-performing students
          students+performance -> ratings from students and their performance in 3 classes
    '''
    QTMat_T = QTMat.T
    if mode == 'students':
        QTMat_extra_T = QTMat_extra.T
        QMAT = np.vstack((QTMat_T, QTMat_extra_T))
        
    elif mode == 'students+performance':
        QTMat_extra_T = QTMat_extra.T
        QMAT = np.vstack((QTMat_T, QTMat_extra_T))
        
        
    train_set = QMAT.copy()
    test_set = np.zeros_like(QMAT)
    for index in indices:
        if index != None:
            if mode == 'students+performance':
                train_set[-1, index] = 0
                train_set[-2, index] = 0
                train_set[-3, index] = 0
                test_set[-1, index] = QMAT[-1, index]
                test_set[-2, index] = QMAT[-2, index]
                test_set[-3, index] = QMAT[-3, index]  
            else:
                train_set[-1, index] = 0
                test_set[-1, index] = QMAT[-1, index]
    WriteMatrixToTable(recomfolder, "train.csv", train_set, uDict, qDict,[])
    WriteMatrixToTable(recomfolder, "test.csv", test_set, uDict, qDict,[])
    return test_set


def MF_cv(QTMat, QTMat_extra, QRMat, uDict, uSize, qDict, qSize, add_info, mode):
    random.seed(0)
    rmse_tot = []
    indices = np.arange(42) # selecting 9 indices from expert row for the test set
    indices
    indices = np.append(indices, [None]*8)
    random.shuffle(indices)
    indices = np.reshape(indices, (-1, 5))
    if mode == 'students+performance':
        MF_predictions = np.zeros((qSize,3))
        ground_truth = np.zeros((qSize, 1))
        MF_predictions_list_h = [] 
        MF_predictions_list_avg = []
        MF_predictions_list_l = []
        ground_truth_list = []

        for i in np.arange(len(indices)):
            test = spilt_data_cv(QTMat, QTMat_extra, QRMat, uDict, qDict, indices[i], mode)
            RecSys(recomfolder, recomfolder, recommender,trainFile, testFile, predictionFile)
            Rprime = load(recomfolder, predictionFile, uDict, uSize, add_info, qDict, qSize, "	")
            for index in indices[i]:
                if index != None:
                    ground_truth[index] = QRMat[index]
                    ground_truth_list.append(QRMat[index])
                    MF_predictions[index, 0] = Rprime[-3, index]
                    MF_predictions[index, 1] = Rprime[-2, index]
                    MF_predictions[index, 2] = Rprime[-1, index]
                    MF_predictions_list_h.append(Rprime[-3, index])
                    MF_predictions_list_avg.append(Rprime[-2, index])
                    MF_predictions_list_l.append(Rprime[-1, index])
            rmse_h = sqrt(mean_squared_error(MF_predictions_list_h, ground_truth_list))
            rmse_avg = sqrt(mean_squared_error(MF_predictions_list_avg, ground_truth_list))
            rmse_l = sqrt(mean_squared_error(MF_predictions_list_l, ground_truth_list))
            rmse_tot.append([rmse_h, rmse_avg, rmse_l])
            MF_predictions_list_h = [] 
            MF_predictions_list_avg = []
            MF_predictions_list_l = []
            ground_truth_list = []

        for i in np.arange(3):
            rmse = sqrt(mean_squared_error(list(MF_predictions[:, i]), list(ground_truth)))
        
    else:
        MF_predictions = np.zeros((qSize))
        ground_truth = np.zeros((qSize))
        MF_predictions_list = [] 
        ground_truth_list = []
        for i in np.arange(len(indices)):
            test = spilt_data_cv(QTMat, QTMat_extra, QRMat, uDict, qDict, indices[i], mode)
            RecSys(recomfolder, recomfolder, recommender,trainFile, testFile, predictionFile)
            Rprime = load(recomfolder, predictionFile, uDict, uSize, add_info, qDict, qSize, "	")
            for index in indices[i]:
                if index != None:
                    ground_truth[index] = QRMat[index]
                    ground_truth_list.append(QRMat[index])
                    MF_predictions_list.append(Rprime[-1, index])
                    MF_predictions[index] = Rprime[-1, index]
            rmse_tot.append(sqrt(mean_squared_error((MF_predictions_list), (ground_truth_list))))
            MF_predictions_list = [] 
            ground_truth_list = []

    return rmse_tot

## Evaluations

#### Students

In [ ]:
rmse_app1 = MF_cv(QTMat, QTMat_mean, QRMat, uDict, uSize, qDict, qSize, 1, 'students')

In [ ]:
# rmse and std for the class based on the first MF-based approach
rmse_app1_students = np.mean(rmse_app1)
prvar(rmse_app1_students)

std_app1_students = np.std(rmse_app1)
prvar(std_app1_students)

#### Students+Performance

In [ ]:
# in the old version, mode should be 'student+performance'. This model predecits the rating from 
# each performance group of student point of view. The current version, predicts ratings from 
# high-perofrming students point of view
rmse_app2 = MF_cv(QTMat, QTMat_reputation, QRMat, uDict, uSize, qDict, qSize,1, 'students')

In [ ]:
rmse_app2_students = np.mean(rmse_app2)
prvar(rmse_app2_students)

std_app2_students = np.std(rmse_app2)
prvar(std_app2_students)

# if mode is 'student+performance'
# rmse_app2 = np.array(rmse_app2)

# # rmse and std for high-performing
# rmse_app2_hp = np.mean(rmse_app2[:, 0])
# std_app2_hp = np.std(rmse_app2[:,0])
# prvar(rmse_app2_hp)

# #rmse and std for average-performing
# rmse_app2_ap = np.mean(rmse_app2[:, 1])
# std_app2_ap = np.std(rmse_app2[:,1])
# prvar(rmse_app2_ap)

# #rmse and std for low-performing
# rmse_app2_lp = np.mean(rmse_app2[:, 2])
# std_app2_lp = np.std(rmse_app2[:,2])
# prvar(rmse_app2_lp)


# The two semi-supervised MF-base approaches

In [ ]:
def spilt_data_cv_density(QTMat, QTMat_extra,  QRMat, uDict, qDict, indices, train_idx, mode):
    '''
    mode: students+experts -> ratings from students and experts ratings
          students+performance+experts -> ratings from students, their performance and ratings from experts
    ''' 
    QTMat_T = QTMat.T
    QRMat_T = QRMat.T
    QRMat_zeros_T = np.zeros_like(QRMat_T)
    QRMat_zeros_T[0, train_idx] = QRMat_T[0, train_idx]
    if mode == 'students+experts':
        QMAT = np.vstack((QTMat_T,QRMat_zeros_T))
    else:
        QTMat_extra_T = QTMat_extra.T
        QMAT = np.vstack((QTMat_T,QTMat_extra_T, QRMat_zeros_T))
    
    train_set = QMAT.copy()
    test_set = np.zeros_like(QMAT)
    for index in indices:
        if index != None:
            test_set[-1, index] = QRMat_T[0, index]
    WriteMatrixToTable(recomfolder, "train.csv", train_set, uDict, qDict,[])
    WriteMatrixToTable(recomfolder, "test.csv", test_set, uDict, qDict,[])
    return test_set


def MF_cv_density(QTMat, QTMat_extra, QRMat, uDict, uSize, qDict, qSize, add_info, mode):
    random.seed(0)
    rmse_tot = []
    indices = np.arange(42) # selecting 9 indices from expert row for the test set
    indices = np.append(indices, [None]*8) # adding 8 none values to the indices to make them dividable by 8
    random.shuffle(indices)
    indices = np.reshape(indices, (-1, 5)) # rearange indices into a 10 by 5 where values in each row determines indices to be in the test set
        
    MF_predictions = np.zeros((qSize))
    ground_truth = np.zeros((qSize))
    MF_predictions_list = [] 
    ground_truth_list = []
    
    for i in np.arange(len(indices)):
        none_cntr = np.count_nonzero(indices[i] !=None) #counting the number of none values for that specific fold
        test_fold_len = len(indices[i])
        train_index = [x for x in np.arange(42) if x not in indices[i]] #indices that are not in the test fold and can be used in training
        train_len = [2, 4, 6, 8]
        for j in np.arange(len(train_len)):
            train_indices = random.sample(train_index, train_len[j]) #j)
            test = spilt_data_cv_density(QTMat, QTMat_extra, QRMat, uDict, qDict, indices[i], train_indices, mode)
            RecSys(recomfolder, recomfolder, recommender,trainFile, testFile, predictionFile)
            Rprime = load(recomfolder, predictionFile, uDict, uSize, add_info, qDict, qSize, "	")
            
            for index in indices[i]:
                if index != None:
                    ground_truth[index] = QRMat[index]
                    ground_truth_list.append(QRMat[index])
                    MF_predictions_list.append(Rprime[-1, index])
                    MF_predictions[index] = Rprime[-1, index]
                    
#                     prvar([j, sqrt(mean_squared_error((MF_predictions_list), (ground_truth_list)))])
            if (j != (len(train_len)-1)):
                rmse_tot.append([train_len[j], sqrt(mean_squared_error((MF_predictions_list), (ground_truth_list)))])
            else:
                rmse_tot.append([37, sqrt(mean_squared_error((MF_predictions_list), (ground_truth_list)))])
                
            MF_predictions_list = [] 
            ground_truth_list = []
            
    rmse_tot_df = pd.DataFrame(rmse_tot, columns=['density', 'RMSE'])
    RMSE_mean = rmse_tot_df.groupby(['density'])['RMSE'].mean()
    return RMSE_mean

### Student+Performance+Expert

In [ ]:
rmse_app3 = MF_cv_density(QTMat, QTMat_reputation,  QRMat, uDict, uSize, qDict, qSize,2, 'students+performance+experts')
prvar(rmse_app3)

### Student + Expert

In [ ]:
rmse_app4 = MF_cv_density(QTMat, None,  QRMat, uDict, uSize, qDict, qSize,1, 'students+experts')
prvar(rmse_app4)